In [48]:
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel, TfidfModel
import numpy as np
import tqdm
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from pprint import pprint

In [2]:
raw_sop = pd.read_csv('./data/sops.csv')

In [3]:
raw_sop.head()

,city,sop_doc
0,Austin,Austin Police DepartmentPolicy ManualCopyright...
1,Austin,Austin Police DepartmentPolicy ManualCopyright...
2,Austin,Austin Police DepartmentPolicy ManualCopyright...
3,Austin,Austin Police DepartmentPolicy ManualCanons of...
4,Austin,Austin Police DepartmentPolicy ManualCanons of...


In [4]:
sop = raw_sop.drop(columns=['city'])

In [5]:
sop.sample(10)

,sop_doc
734,Austin Police DepartmentPolicy ManualPurchasin...
4773,DRIVING UNDER THE INFLUENCE INVESTIGATIONS\n O...
1715,HOME\n ADDRESS\n This\n field\n is used\n to i...
273,Austin Police DepartmentPolicy ManualPrelimina...
5093,Professional Conduct and Ethics\n still requi...
2686,Evanston Police Department\nPolicy Manual\nPer...
3183,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
175,Austin Police DepartmentPolicy ManualArrestsCo...
6190,Las Vegas Metropolitan Police DepartmentPartne...
481,Austin Police DepartmentPolicy ManualEquipment...


In [6]:
sop['sop_doc_processed'] = sop[
    'sop_doc'
].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<ipython-input-6-da76bbe04fda>:3: DeprecationWarning: invalid escape sequence \.
  ].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))


In [7]:
sop['sop_doc_processed'] = sop['sop_doc_processed'].map(lambda x: x.lower().strip())

In [8]:
sop = sop[sop['sop_doc_processed'] != '']

In [9]:
sop.sample(10)

,sop_doc,sop_doc_processed
893,Dallas Police Department General Order\n \n \n...,dallas police department general order 321...
2776,Evanston Police Department\nPolicy Manual\nLin...,evanston police department policy manual line-...
240,Austin Police DepartmentPolicy ManualCrash Inv...,austin police departmentpolicy manualcrash inv...
1978,The video is stored in the Exacq interview roo...,the video is stored in the exacq interview roo...
5779,Staff Support\n WHEN SUSPECT KNOWN\n: When a...,staff support when suspect known : when a s...
2068,Policy\n302\nEvanston Police Department\nPolic...,policy 302 evanston police department policy m...
4996,Administrative Structure\n - Conducts investi...,administrative structure - conducts investig...
777,\n \nDallas Police \nDepartment General Order...,dallas police department general order ...
5474,Line/Operation\ns Procedures\n L 5805 RESPONS...,line/operation s procedures l 5805 responsib...
4606,DONATIONS/CONTRIBUTIONS\n Operations Order\n 4...,donations/contributions operations order 429...


In [16]:
sop.shape

(4883, 2)

In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [11]:
data = sop.sop_doc_processed.values.tolist()
data_words = np.array(list(sent_to_words(data)))

In [18]:
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=1) # higher threshold fewer phrases.

In [21]:
trigram = gensim.models.Phrases(bigram[data_words], threshold=1)

In [23]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [24]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [26]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [28]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([
            token.lemma_ for token in doc if token.pos_ in allowed_postags
        ])
    return texts_out

In [29]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [30]:
print(data_lemmatized[:1])

[['police', 'departmentchief', 'message', 'chief', 'proud', 'present', 'new', 'edition', 'policy', 'manual', 'design', 'assist', 'employee', 'accomplish', 'mission', 'lawful', 'manner', 'adherence', 'policy', 'also', 'help', 'safeguard', 'employee', 'ensure', 'employee', 'protect', 'individual', 'action', 'scrutinize', 'especially', 'critical', 'incidentall', 'employee', 'abide', 'policy', 'responsible', 'keep', 'current', 'content', 'safebrianbrian', 'manleychief', 'police']]


In [51]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Tfidf corpus
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [64]:
hdp_model_bow = gensim.models.HdpModel(
    corpus=corpus,
    id2word=id2word
)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [65]:
hdp_model_tfidf = gensim.models.HdpModel(
    corpus=corpus_tfidf,
    id2word=id2word
)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [63]:
pprint(hdp_model_bow.print_topics())

[(0,
  '0.002*vehicle + 0.002*member + 0.002*officer + 0.002*employee + '
  '0.002*report + 0.002*victim + 0.001*suspect + 0.001*shall + 0.001*unit + '
  '0.001*information'),
 (1,
  '0.001*victim + 0.001*shall + 0.001*employee + 0.001*vehicle + 0.001*member '
  '+ 0.001*officer + 0.001*report + 0.001*person + 0.001*information + '
  '0.001*form'),
 (2,
  '0.001*member + 0.001*employee + 0.001*vehicle + 0.001*report + 0.001*arrest '
  '+ 0.001*training + 0.001*officer + 0.001*shall + 0.001*form + 0.001*person'),
 (3,
  '0.001*employee + 0.001*member + 0.000*hold + 0.000*officer + '
  '0.000*condition + 0.000*incident + 0.000*explode + 0.000*property + '
  '0.000*award + 0.000*helpfulness'),
 (4,
  '0.001*vehicle + 0.001*shall + 0.000*employee + 0.000*arrest + 0.000*agency '
  '+ 0.000*ilcs + 0.000*must + 0.000*ntion + 0.000*releasable + 0.000*formal'),
 (5,
  '0.001*vehicle + 0.001*report + 0.000*employee + 0.000*member + 0.000*beer + '
  '0.000*duty + 0.000*action + 0.000*complaint + 

In [66]:
pprint(hdp_model_tfidf.print_topics())

[(0,
  '0.003*member + 0.003*vehicle + 0.002*employee + 0.002*officer + '
  '0.002*report + 0.002*shall + 0.002*victim + 0.002*person + '
  '0.002*information + 0.002*training'),
 (1,
  '0.001*thecase + 0.000*documenting + 0.000*objectively_reasonable + '
  '0.000*cancell + 0.000*employee + 0.000*establis + 0.000*valuesbehavior + '
  '0.000*ponytail + 0.000*service + 0.000*cleanser'),
 (2,
  '0.000*forgo + 0.000*acitizen + 0.000*increase + 0.000*iad + 0.000*record + '
  '0.000*analyst + 0.000*partly + 0.000*immigration + 0.000*irregular + '
  '0.000*preference'),
 (3,
  '0.001*arrive + 0.000*departmentbody + 0.000*requireman + 0.000*employ + '
  '0.000*space + 0.000*trc + 0.000*chief + 0.000*inuance + 0.000*aobtain + '
  '0.000*controlling'),
 (4,
  '0.000*trangulation + 0.000*proper + 0.000*gear + 0.000*perspective + '
  '0.000*transmitter + 0.000*moist + 0.000*holster + 0.000*nthly + '
  '0.000*disclosable + 0.000*orinitiate'),
 (5,
  '0.000*distinct + 0.000*ﬁnee + 0.000*noticeﬂ + 0.

In [71]:
coherence_model_lda_bow = CoherenceModel(model=hdp_model_bow, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [72]:
coherence_model_lda_bow.get_coherence()

0.5468136911806611

In [73]:
coherence_model_lda_tfidf = CoherenceModel(model=hdp_model_tfidf, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [74]:
coherence_model_lda_tfidf.get_coherence()

0.7102316986206483